In [1]:
%matplotlib inline

In [2]:
from Bio import SeqIO, Phylo
import glob
import random

import pandas as pd
import numpy as np

# Just working with OMA groups first

In [66]:
monophyly_df = pd.read_csv('../Data/OMA_group_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

(400, 9)


,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
OMAGroup_789494,IntNode_3,IntNode_88,1.49293,69.00150,56.46590,126.96033,194.0,89.0,105.0
OMAGroup_555430,IntNode_46,IntNode_73,1.00116,30.16103,38.54124,69.70343,222.0,94.0,128.0
OMAGroup_828657,IntNode_43,IntNode_49,0.97504,16.43967,28.31956,45.73427,172.0,50.0,122.0
OMAGroup_804387,IntNode_48,IntNode_76,0.37875,20.59071,17.27605,38.24551,233.0,97.0,136.0
OMAGroup_809161,IntNode_3,IntNode_95,3.04869,32.59700,28.92590,64.57159,213.0,96.0,117.0


# Preserving the exact root and downstream clades

Initially, I ran this code to prune species in such a way as to **always** preserve the length of the root node. Which is to say, when pruning a single side/set I would ensure that the first bifurcation remained no matter what. Thus, I'd make sure to preserve at least two species on any given side when pruning so that this first split never "moved" and that the length of the target root sequence would always be the same no matter the pruning method.

In [17]:
# target_mono_frac = 0.95
# for tree_loc in glob.glob('../Data/OMA_group_data/eukaryotes/'
#                           'processed_OMA_trees/*.RootedNoZero.MPAJH')[:]:
#     file_name = tree_loc.split('/')[-1]
#     print(file_name)
#     group = tree_loc.split('/')[-1].split('.')[0]
#     if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
#         print('not mono')
#         continue
#     starting_tree = Phylo.read(tree_loc, 'newick')
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
#     assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
#     assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
#     initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     ########
#     mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
#     assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
#     other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
#     assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
#     total_terms = len(mono_ids)+len(other_ids)
#     starting_mono_frac = len(mono_ids)/total_terms
#     if starting_mono_frac >= target_mono_frac:
#         clade_to_trim_from = 1
#         n_to_retain = target_mono_frac*total_terms
#         n_to_remove = round(len(mono_ids)-n_to_retain)
#     elif starting_mono_frac < target_mono_frac:
#         clade_to_trim_from = 0
#         n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
#         n_to_remove = round(len(other_ids)-n_to_retain)
#     if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
#         print('With given parameters, you would remove all from clade. Leaving one member')
#         n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

#     ###I'm making this overly complicated here so that I don't ever accidentally remove the 
#     ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
#     ###removal of a single clade connected to the monophyletic node
#     for i in range(n_to_remove):
#         #######
#         l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
#         r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
#         #######
#         if len(l_terms) > 1 and len(r_terms) > 1:
#             combined_terms = l_terms + r_terms
#         elif len(l_terms) > 1 and len(r_terms) == 1:
#             combined_terms = l_terms
#         elif len(l_terms) == 1 and len(r_terms) > 1:
#             combined_terms = r_terms
#         else:
#             print('Something unexpected is happening during node removal and you should investigate')
#             print(len(l_terms), len(r_terms))
#             break
#         term_to_remove = random.choice(combined_terms)
#         starting_tree.prune(term_to_remove)
#     final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     assert initial_root_dist == final_root_dist
#     outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
#     outfile_loc = outfile_loc.replace(file_name, 
#                                       '{}.frac{}.treefile'.format(group, target_mono_frac))
#     Phylo.write(starting_tree, outfile_loc, 'newick')

# Removing the restriction

Now, this code will remove any/all species from a given side without regard to who/the first bifurcation

In [69]:
target_mono_frac = 0.9
for tree_loc in glob.glob('../Data/OMA_group_data/eukaryotes/'
                          'processed_OMA_trees/*.RootedNoZero.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = round((target_mono_frac*len(other_ids))/(1-target_mono_frac))
        n_to_remove = round(len(mono_ids)-n_to_retain)
    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = round((len(mono_ids) - (target_mono_frac*len(mono_ids)))/target_mono_frac)
        n_to_remove = round(len(other_ids)-n_to_retain)
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    for i in range(n_to_remove):
        my_terms = starting_tree.root.clades[clade_to_trim_from].get_terminals()
        term_to_remove = random.choice(my_terms)
        starting_tree.prune(term_to_remove)
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    outfile_loc = tree_loc.replace('processed_OMA_trees/', 'pruned_OMA_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    print(len(starting_tree.root.clades[0].get_terminals())/len(starting_tree.get_terminals()))
    print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
    Phylo.write(starting_tree, outfile_loc, 'newick')

OMAGroup_789494.treefile.RootedNoZero.MPAJH
0.10101010101010101
0.898989898989899
OMAGroup_555430.treefile.RootedNoZero.MPAJH
0.09615384615384616
0.9038461538461539
OMAGroup_828657.treefile.RootedNoZero.MPAJH
0.10714285714285714
0.8928571428571429
OMAGroup_804387.treefile.RootedNoZero.MPAJH
0.10185185185185185
0.8981481481481481
OMAGroup_809161.treefile.RootedNoZero.MPAJH
0.102803738317757
0.897196261682243
OMAGroup_839524.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
OMAGroup_840929.treefile.RootedNoZero.MPAJH
0.10204081632653061
0.8979591836734694
OMAGroup_677247.treefile.RootedNoZero.MPAJH
0.10101010101010101
0.898989898989899
OMAGroup_692878.treefile.RootedNoZero.MPAJH
0.0970873786407767
0.9029126213592233
OMAGroup_696620.treefile.RootedNoZero.MPAJH
0.0975609756097561
0.9024390243902439
OMAGroup_485746.treefile.RootedNoZero.MPAJH
0.0970873786407767
0.9029126213592233
OMAGroup_649311.treefile.RootedNoZero.MPAJH
not mono
OMAGroup_813875.treefile.RootedNoZero.MPAJ

# And a similar thing for the Tria *et al.* dataset 

In [18]:
monophyly_df = pd.read_csv('../Data/Tria_et_al_data/eukaryotes/tree_monophyly.csv', index_col='Group_id')
print(monophyly_df.shape)
monophyly_df.head()

(400, 9)


,monophyletic_clade,other_clade,root_bl,monophyletic_total_bl,other_total_bl,total_tree_bl,total_n,monophyletic_n,other_n
Group_id,,,,,,,,,
KOG2115,IntNode_3,IntNode_4,2.16480,11.65634,8.83701,22.65815,31.0,17.0,14.0
KOG0266,IntNode_13,IntNode_10,1.18180,5.44034,15.87080,22.49294,31.0,17.0,14.0
KOG2609,IntNode_3,IntNode_16,1.27520,18.11668,7.23651,26.62839,31.0,17.0,14.0
KOG0800,IntNode_11,IntNode_10,1.28320,8.93623,20.26973,30.48916,31.0,17.0,14.0
KOG1506,IntNode_13,IntNode_6,0.38028,2.31323,2.37654,5.07005,31.0,17.0,14.0


# The old way...

In [50]:
# target_mono_frac = 0.9
# for tree_loc in glob.glob('../Data/Tria_et_al_data/eukaryotes/'
#                           'processed_trees/*.RootedNoZero.MPAJH')[:]:
#     file_name = tree_loc.split('/')[-1]
#     print(file_name)
#     group = tree_loc.split('/')[-1].split('.')[0]
#     if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
#         print('not mono')
#         continue
#     starting_tree = Phylo.read(tree_loc, 'newick')
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
#     starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
#     assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
#     assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
#     initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     ########
#     mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
#     assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
#     other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
#     assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
#     total_terms = len(mono_ids)+len(other_ids)
#     starting_mono_frac = len(mono_ids)/total_terms
#     if starting_mono_frac >= target_mono_frac:
#         clade_to_trim_from = 1
#         n_to_retain = target_mono_frac*total_terms
#         n_to_remove = round(len(mono_ids)-n_to_retain)
#     elif starting_mono_frac < target_mono_frac:
#         clade_to_trim_from = 0
#         n_to_retain = len(mono_ids)/target_mono_frac - len(mono_ids)
#         n_to_remove = round(len(other_ids)-n_to_retain)
#     if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
#         print('With given parameters, you would remove all from clade. Leaving one member')
#         n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

#     ###I'm making this overly complicated here so that I don't ever accidentally remove the 
#     ###monophyletic node entirely! It iteratively removes one at a time and does not allow for
#     ###removal of a single clade connected to the monophyletic node
#     for i in range(n_to_remove):
#         #######
#         l_terms = starting_tree.root.clades[clade_to_trim_from].clades[0].get_terminals()
#         r_terms = starting_tree.root.clades[clade_to_trim_from].clades[1].get_terminals()
#         #######
#         if len(l_terms) > 1 and len(r_terms) > 1:
#             combined_terms = l_terms + r_terms
#         elif len(l_terms) > 1 and len(r_terms) == 1:
#             combined_terms = l_terms
#         elif len(l_terms) == 1 and len(r_terms) > 1:
#             combined_terms = r_terms
#         else:
#             print('Something unexpected is happening during node removal and you should investigate')
#             print(len(l_terms), len(r_terms))
#             break
#         term_to_remove = random.choice(combined_terms)
#         starting_tree.prune(term_to_remove)
#     final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
#     assert initial_root_dist == final_root_dist
#     outfile_loc = tree_loc.replace('processed_trees/', 'pruned_trees/')
#     outfile_loc = outfile_loc.replace(file_name, 
#                                       '{}.frac{}.treefile'.format(group, target_mono_frac))
#     print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
#     Phylo.write(starting_tree, outfile_loc, 'newick')

KOG2115.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG0266.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2609.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG0800.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG1506.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG4205.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG3675.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG4103.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2496.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG3169.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2002.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG0371.treefile.RootedNoZero.MPAJH
not mono
KOG3887.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2581.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG1411.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2759.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG1651.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG0176.treefile.RootedNoZero.MPAJH
0.8947368421052632
KOG2519.treefile.Roo

# And the new way

dumb dumb dumb dumb

In [64]:
target_mono_frac = 0.9
for tree_loc in glob.glob('../Data/Tria_et_al_data/eukaryotes/'
                          'processed_trees/*.RootedNoZero.MPAJH')[:]:
    file_name = tree_loc.split('/')[-1]
    print(file_name)
    group = tree_loc.split('/')[-1].split('.')[0]
    if type(monophyly_df.loc[group]['monophyletic_clade']) != str:
        print('not mono')
        continue
    starting_tree = Phylo.read(tree_loc, 'newick')
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['other_clade'], outgroup_branch_length=0.0)
    starting_tree.root_with_outgroup(monophyly_df.loc[group]['monophyletic_clade'], outgroup_branch_length=10e-10)
    assert starting_tree.root.clades[1].name == monophyly_df.loc[group]['monophyletic_clade']
    assert starting_tree.root.clades[0].name == monophyly_df.loc[group]['other_clade']
    initial_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    ########
    mono_ids = [term.name for term in starting_tree.root.clades[1].get_terminals()]
    assert len(mono_ids) == len(starting_tree.root.clades[1].get_terminals())
    other_ids = [term.name for term in starting_tree.root.clades[0].get_terminals()]
    assert len(other_ids) == len(starting_tree.root.clades[0].get_terminals())
    
    total_terms = len(mono_ids)+len(other_ids)
    starting_mono_frac = len(mono_ids)/total_terms
    if starting_mono_frac >= target_mono_frac:
        clade_to_trim_from = 1
        n_to_retain = round((target_mono_frac*len(other_ids))/(1-target_mono_frac))
        n_to_remove = len(mono_ids)-n_to_retain


    elif starting_mono_frac < target_mono_frac:
        clade_to_trim_from = 0
        n_to_retain = round((len(mono_ids) - (target_mono_frac*len(mono_ids)))/target_mono_frac)
        n_to_remove = round(len(other_ids)-n_to_retain)
        
    if n_to_remove >= len(starting_tree.root.clades[clade_to_trim_from].get_terminals()):
        print('With given parameters, you would remove all from clade. Leaving one member')
        n_to_remove = len(starting_tree.root.clades[clade_to_trim_from].get_terminals())-1

    for i in range(n_to_remove):
        my_terms = starting_tree.root.clades[clade_to_trim_from].get_terminals()
        term_to_remove = random.choice(my_terms)
        starting_tree.prune(term_to_remove)
        
    final_root_dist = [clade.branch_length for clade in starting_tree.root.clades]
    outfile_loc = tree_loc.replace('processed_trees/', 'pruned_trees/')
    outfile_loc = outfile_loc.replace(file_name, 
                                      '{}.frac{}.treefile'.format(group, target_mono_frac))
    print(len(starting_tree.root.clades[0].get_terminals())/len(starting_tree.get_terminals()))
    print(len(starting_tree.root.clades[1].get_terminals())/len(starting_tree.get_terminals()))
    Phylo.write(starting_tree, outfile_loc, 'newick')

KOG2115.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG0266.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG2609.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG0800.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG1506.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG4205.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG3675.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG4103.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG2496.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG3169.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG2002.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG0371.treefile.RootedNoZero.MPAJH
not mono
KOG3887.treefile.RootedNoZero.MPAJH
0.10526315789473684
0.8947368421052632
KOG2581.treefile.RootedNoZero.MPAJH
0.10526315789473684

1

In [34]:
1/16

0.0625